In [ ]:
# default_exp core

# core

> Core utilities for working with DICOM files. Utilizes the `pydicom` and `fastcore` packages. Some ideas borrowed from [fastai.medical.image](https://github.com/fastai/fastai/blob/master/fastai/medical/imaging.py).

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
import pydicom

from fastcore.all import *
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
from typing import Union

In [ ]:
#export
study_tags = {
    'Study Instance UID': (0x0020, 0x000d),
    'Series Instance UID': (0x0020, 0x000e),
    'SOP Class UID': (0x0008, 0x0016), # e.g. "MR Image Storage"
    'SOP Instance UID': (0x0008, 0x0018),
    'Study Description': (0x0008, 0x1030),
    'Study Date': (0x0008, 0x0020),
    'Series Description': (0x0008, 0x103e),
    'Patient ID': (0x0010, 0x0020),
    'Series Number': (0x0020, 0x0011),
    'Instance Number': (0x0020, 0x0013),
    'Window Center': (0x0028, 0x1050),
    'Window Level': (0x0028, 0x1051)
}

In [ ]:
#export
@patch
def dcmread(fn: Path, no_pixels=True, force=True):
    "Reads a DICOM file and returns the corresponding pydicom.Dataset"
    return pydicom.dcmread(str(fn), stop_before_pixels=no_pixels, force=force)

In [ ]:
#export
def _cast_dicom_special(x):
    cls = type(x)
    if not cls.__module__.startswith('pydicom'): return x
    if cls.__base__ == object: return x
    return cls.__base__(x)

def _split_elem(res, k, v):
    if not isinstance(v, DcmMultiValue): return
    res[f'Multi{k}'] = 1
    for i, o in enumerate(v): res[f'{k}{"" if i == 0 else i}'] = o

In [ ]:
#export
@patch
def as_dict(self: DcmDataset):
    vals = [self[o] for o in self.keys() if (o in study_tags.values() or o in self.group_dataset(0x0018).keys())]
    items = [(v.keyword, v.value.name) if v.keyword == 'SOPClassUID' else (v.keyword, v.value) for v in vals]
    res = dict(items)
    res['fname'] = self.filename
    for k, v in items: _split_elem(res, k, v)
    for k in res: res[k] = _cast_dicom_special(res[k])
    return res

In [ ]:
#export
def _dcm2dict(fn, incl_private=False, **kwargs):
    if incl_private:
        return fn.dcmread(**kwargs).as_dict()
    else:
        ds = fn.dcmread(**kwargs)
        ds.remove_private_tags()
        return ds.as_dict()

In [ ]:
#export
@delegates(parallel)
def _from_dicoms(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(_dcm2dict, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms = classmethod(_from_dicoms)